In [1]:
from sparknlp.base import *
from sparknlp.annotator import *
import sparknlp

In [2]:
spark = sparknlp.start()

In [3]:
bert = BertEmbeddings.pretrained("bert_multi_cased", "xx") \
      .setInputCols("sentence", "token") \
      .setOutputCol("bert")

bert_multi_cased download started this may take some time.
Approximate size to download 638.7 MB
[OK!]


In [76]:
bert2 = BertEmbeddings.pretrained("bert_multi_cased", "xx")
bert2[101]

bert_multi_cased download started this may take some time.
Approximate size to download 638.7 MB
[OK!]


TypeError: 'BertEmbeddings' object is not subscriptable

In [81]:
bert2.getDimension()

768

In [82]:
bert2.explainParams()

"batchSize: Batch size. Large values allows faster processing but requires more memory. (default: 32)\ncaseSensitive: whether to ignore case in tokens for embeddings matching (default: True, current: True)\nconfigProtoBytes: ConfigProto from tensorflow, serialized into byte array. Get with config_proto.SerializeToString() (undefined)\ndimension: Number of embedding dimensions (default: 768, current: 768)\ninputCols: previous annotations columns, if renamed (current: ['sentence', 'token'])\nlazyAnnotator: Whether this AnnotatorModel acts as lazy in RecursivePipelines (default: False)\nmaxSentenceLength: Max sentence length to process (default: 128)\noutputCol: output annotation column. can be left default. (current: bert)\npoolingLayer: Set BERT pooling layer to: -1 for last hidden layer, -2 for second-to-last hidden layer, and 0 for first layer which is called embeddings (default: 0)\nstorageRef: unique reference name for identification (current: bert_multi_cased)"

In [ ]:
bert2.pretrained()

bert_base_cased download started this may take some time.
Approximate size to download 389.2 MB
[ — ]

In [91]:
bert2.getOrDefault(bert2.vocabulary)

AttributeError: 'BertEmbeddings' object has no attribute 'vocabulary'

In [63]:
type(bert)

sparknlp.annotator.BertEmbeddings

In [64]:
bert.unpack()

AttributeError: 'BertEmbeddings' object has no attribute 'unpack'

In [6]:

with open("/media/iseratho/Movies HD/training1m.tsv") as f:
    training1m = f.readlines()

In [13]:
from pyspark.sql.types import StringType

df = spark.createDataFrame(training1m, StringType()).toDF("text")

In [15]:
df.head(n=5)

[Row(text='101\t4378\t4348\t1946\t3192\t1947\t3293\t1946\t3197\t3365\t1946\t2731\t7762\t1912\t4938\t40710\t11312\t4501\t1938\t18825\t27843\t15802\t60421\t21462\t60421\t21462\t14803\t27058\t16923\t1882\t4146\t8224\t6712\t4004\t104386\t56910\t2179\t1912\t4501\t43753\t35272\t11662\t4965\t8335\t3999\t2035\t22396\t111823\t22396\t1882\t2023\t15970\t30263\t13291\t37133\t30884\t111806\t6242\t8782\t28798\t1882\t3388\t4470\t5718\t1943\t7231\t1911\t16838\t91576\t18767\t26164\t18767\t6237\t4097\t100\t102\x01\x013C07C7EDBF91457F165405541CC63B8A\x01\x01\x01\x01TopLevel\x0122C448FF81263D4BAF2A176145EE9EAD\x011581264609\x0190537F3D3B8DBB17922A945ED4FD3B5A\x01129\x01260\x01false\x011402980271\x015BCEFB026703939F22AF56BB6DA55DE7\x01270\x01253\x01false\x011358823496\x01true\x01\x01\x01\x01\n'),
 Row(text='101\t13209\t18121\t12989\t12056\t10119\t14316\t193\t112\t169\t17828\t10119\t92049\t10460\t44928\t10131\t10119\t92049\t13115\t44928\t117\t20795\t10154\t60727\t10246\t13055\t21542\t180\t100\t110055\t172\t

In [36]:
all_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type","language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "engaging_user_id", "engaging_user_follower_count", "engaging_user_following_count", "engaging_user_is_verified",\
               "engaging_user_account_creation", "engagee_follows_engager", "reply_timestamp", "retweet_timestamp", "retweet_with_comment_timestamp", "like_timestamp"]
# {"reply_timestamp": 20, "retweet_timestamp": 21, "retweet_with_comment_timestamp": 22, "like_timestamp": 23}

In [23]:
df = spark.read.format("com.databricks.spark.csv").option("delimiter", "\x01").load("/media/iseratho/Movies HD/training1m.tsv")

In [37]:
df = df.toDF(*all_features)

In [38]:
df.head(n=5)

[Row(text_tokens='101\t4378\t4348\t1946\t3192\t1947\t3293\t1946\t3197\t3365\t1946\t2731\t7762\t1912\t4938\t40710\t11312\t4501\t1938\t18825\t27843\t15802\t60421\t21462\t60421\t21462\t14803\t27058\t16923\t1882\t4146\t8224\t6712\t4004\t104386\t56910\t2179\t1912\t4501\t43753\t35272\t11662\t4965\t8335\t3999\t2035\t22396\t111823\t22396\t1882\t2023\t15970\t30263\t13291\t37133\t30884\t111806\t6242\t8782\t28798\t1882\t3388\t4470\t5718\t1943\t7231\t1911\t16838\t91576\t18767\t26164\t18767\t6237\t4097\t100\t102', hashtags=None, tweet_id='3C07C7EDBF91457F165405541CC63B8A', present_media=None, present_links=None, present_domains=None, tweet_type='TopLevel', language='22C448FF81263D4BAF2A176145EE9EAD', tweet_timestamp='1581264609', engaged_with_user_id='90537F3D3B8DBB17922A945ED4FD3B5A', engaged_with_user_follower_count='129', engaged_with_user_following_count='260', engaged_with_user_is_verified='false', engaged_with_user_account_creation='1402980271', engaging_user_id='5BCEFB026703939F22AF56BB6DA55

In [43]:
df.select("text_tokens").head(n=5)

[Row(text_tokens='101\t4378\t4348\t1946\t3192\t1947\t3293\t1946\t3197\t3365\t1946\t2731\t7762\t1912\t4938\t40710\t11312\t4501\t1938\t18825\t27843\t15802\t60421\t21462\t60421\t21462\t14803\t27058\t16923\t1882\t4146\t8224\t6712\t4004\t104386\t56910\t2179\t1912\t4501\t43753\t35272\t11662\t4965\t8335\t3999\t2035\t22396\t111823\t22396\t1882\t2023\t15970\t30263\t13291\t37133\t30884\t111806\t6242\t8782\t28798\t1882\t3388\t4470\t5718\t1943\t7231\t1911\t16838\t91576\t18767\t26164\t18767\t6237\t4097\t100\t102'),
 Row(text_tokens='101\t13209\t18121\t12989\t12056\t10119\t14316\t193\t112\t169\t17828\t10119\t92049\t10460\t44928\t10131\t10119\t92049\t13115\t44928\t117\t20795\t10154\t60727\t10246\t13055\t21542\t180\t100\t110055\t172\t112\t10119\t26854\t102'),
 Row(text_tokens='101\t8544\t1913\t22445\t64126\t15802\t101369\t54643\t111830\t4238\t1974\t7721\t10497\t18628\t28224\t27849\t56910\t1881\t2036\t67221\t55052\t51143\t16838\t90230\t16838\t15802\t18825\t60847\t119539\t119539\t37133\t111795\t30884\t1

In [65]:
df.printSchema()

root
 |-- text_tokens: string (nullable = true)
 |-- hashtags: string (nullable = true)
 |-- tweet_id: string (nullable = true)
 |-- present_media: string (nullable = true)
 |-- present_links: string (nullable = true)
 |-- present_domains: string (nullable = true)
 |-- tweet_type: string (nullable = true)
 |-- language: string (nullable = true)
 |-- tweet_timestamp: string (nullable = true)
 |-- engaged_with_user_id: string (nullable = true)
 |-- engaged_with_user_follower_count: string (nullable = true)
 |-- engaged_with_user_following_count: string (nullable = true)
 |-- engaged_with_user_is_verified: string (nullable = true)
 |-- engaged_with_user_account_creation: string (nullable = true)
 |-- engaging_user_id: string (nullable = true)
 |-- engaging_user_follower_count: string (nullable = true)
 |-- engaging_user_following_count: string (nullable = true)
 |-- engaging_user_is_verified: string (nullable = true)
 |-- engaging_user_account_creation: string (nullable = true)
 |-- engag

In [74]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, IntegerType

udf_unbert = udf(lambda text: list(map(int, text.split("\t"))), ArrayType(IntegerType()))
# df.select("text_tokens").map(udf_unbert)
text_tokens_unbert = df.withColumn("text_tokens", udf_unbert("text_tokens"))

In [75]:
text_tokens_unbert.head(n=5)

[Row(text_tokens=[101, 4378, 4348, 1946, 3192, 1947, 3293, 1946, 3197, 3365, 1946, 2731, 7762, 1912, 4938, 40710, 11312, 4501, 1938, 18825, 27843, 15802, 60421, 21462, 60421, 21462, 14803, 27058, 16923, 1882, 4146, 8224, 6712, 4004, 104386, 56910, 2179, 1912, 4501, 43753, 35272, 11662, 4965, 8335, 3999, 2035, 22396, 111823, 22396, 1882, 2023, 15970, 30263, 13291, 37133, 30884, 111806, 6242, 8782, 28798, 1882, 3388, 4470, 5718, 1943, 7231, 1911, 16838, 91576, 18767, 26164, 18767, 6237, 4097, 100, 102], hashtags=None, tweet_id='3C07C7EDBF91457F165405541CC63B8A', present_media=None, present_links=None, present_domains=None, tweet_type='TopLevel', language='22C448FF81263D4BAF2A176145EE9EAD', tweet_timestamp='1581264609', engaged_with_user_id='90537F3D3B8DBB17922A945ED4FD3B5A', engaged_with_user_follower_count='129', engaged_with_user_following_count='260', engaged_with_user_is_verified='false', engaged_with_user_account_creation='1402980271', engaging_user_id='5BCEFB026703939F22AF56BB6DA55

In [71]:
text_tokens_unbert.printSchema()

root
 |-- text_tokens: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- hashtags: string (nullable = true)
 |-- tweet_id: string (nullable = true)
 |-- present_media: string (nullable = true)
 |-- present_links: string (nullable = true)
 |-- present_domains: string (nullable = true)
 |-- tweet_type: string (nullable = true)
 |-- language: string (nullable = true)
 |-- tweet_timestamp: string (nullable = true)
 |-- engaged_with_user_id: string (nullable = true)
 |-- engaged_with_user_follower_count: string (nullable = true)
 |-- engaged_with_user_following_count: string (nullable = true)
 |-- engaged_with_user_is_verified: string (nullable = true)
 |-- engaged_with_user_account_creation: string (nullable = true)
 |-- engaging_user_id: string (nullable = true)
 |-- engaging_user_follower_count: string (nullable = true)
 |-- engaging_user_following_count: string (nullable = true)
 |-- engaging_user_is_verified: string (nullable = true)
 |-- engaging_user_accou